In [1]:
from fins3666.defines import *
from fins3666.utils import load_dataframe, load_market, format_market_data, current_fx_data
from fins3666.portfolio import Portfolio


fx = load_market(os.path.join(DIR_DATA_RAW, 'fx-historical-daily.csv'))
ir = load_dataframe(os.path.join(DIR_DATA_RAW, 'ir-historical.csv'))

startTime = datetime(2001, 1, 1)

fx_start = pd.DataFrame(current_fx_data(fx, startTime))


startingAssets = [
    {'asset': 'USD', 'units': ACCOUNT_SIZE_USD, 'unit_value_USD': 1.0},
    {'asset': 'AUD', 'units': 0,
        'unit_value_USD': 1/fx_start.loc[fx_start['asset'] == 'AUD', 'mid'].values[0]},
    {'asset': 'CAD', 'units': 0,
        'unit_value_USD': 1/fx_start.loc[fx_start['asset'] == 'CAD', 'mid'].values[0]},
    {'asset': 'CHF', 'units': 0,
        'unit_value_USD': 1/fx_start.loc[fx_start['asset'] == 'CHF', 'mid'].values[0]},
    {'asset': 'EUR', 'units': 0,
        'unit_value_USD': 1/fx_start.loc[fx_start['asset'] == 'EUR', 'mid'].values[0]},
    {'asset': 'GBP', 'units': 0,
        'unit_value_USD': 1/fx_start.loc[fx_start['asset'] == 'GBP', 'mid'].values[0]},
    {'asset': 'JPY', 'units': 0,
        'unit_value_USD': 1/fx_start.loc[fx_start['asset'] == 'JPY', 'mid'].values[0]},
    {'asset': 'NZD', 'units': 0,
        'unit_value_USD': 1/fx_start.loc[fx_start['asset'] == 'NZD', 'mid'].values[0]},
    {'asset': 'SEK', 'units': 0,
        'unit_value_USD': 1/fx_start.loc[fx_start['asset'] == 'SEK', 'mid'].values[0]},
    {'asset': 'NOK', 'units': 0,
        'unit_value_USD': 1/fx_start.loc[fx_start['asset'] == 'NOK', 'mid'].values[0]}]

SamplePortfolio = Portfolio(startTime, startingAssets)


specTime = datetime(2020, 12, 9)

formattedData = current_fx_data(fx, specTime)
# print('----------------------------------')
# for jawn in formattedData:
#     for k, v in jawn.items():
#         print(f'{k}: {v}')
#     print('----------------------------------')
dates = pd.date_range(start='2023-01-01', end='2023-01-03', freq='D')

for (idx, date) in enumerate(dates):
    currentData = current_fx_data(fx, date)
    SamplePortfolio.updateMarketData(currentData)
    testBuyOrder = Order(asset='AUD', units=1203.80, currency='USD',
                         order='buy', timestamp=date, order_type='market')
    testSellOrder = Order(asset='AUD', units=1203.80, currency='USD',
                          order='sell', timestamp=date, order_type='market')
    testBuyOrder1 = Order(asset='CHF', units=1203.80, currency='JPY',
                          order='buy', timestamp=date, order_type='market')
    testSellOrder1 = Order(asset='CHF', units=1203.80, currency='JPY',
                           order='sell', timestamp=date, order_type='market')
    SamplePortfolio.new_order(testBuyOrder)
    SamplePortfolio.new_order(testSellOrder)
    SamplePortfolio.new_order(testBuyOrder1)
    SamplePortfolio.new_order(testSellOrder1)

    SamplePortfolio.update(date)
    print(SamplePortfolio)

SyntaxError: invalid syntax (portfolio.py, line 164)

In [ ]:
specTime = '2020-12-09'

currentData = current_fx_data(fx, date)


test = '¥'

print(test)

"""
LEDGER FORMAT
    'OpenTimestamp': pd.Series(dtype='pd.datetime64[ns]'),
    'UpdateTimestamp': pd.Series(dtype='pd.datetime64[ns]'),
    'Asset': pd.Series(dtype='str'),
    'Units': pd.Series(dtype='float64'),
    'UnitWAP': pd.Series(dtype='float64'),
    'TransactionIdxs': pd.Series(dtype='list'),
    'AssetType': pd.Series(dtype='str'),
    'CloseTimestamp': pd.Series(dtype='pd.datetime64[ns]'),
    'Open': pd.Series(dtype='Bool')})

TRADE FORMAT

    'Timestamp': pd.Series(dtype='pd.datetime64[ns]'),
    'Asset': pd.Series(dtype='str'),
    'Units': pd.Series(dtype='float64'),
    'UnitPriceUSD': pd.Series(dtype='float64'),
    'OrderValueUSD': pd.Series(dtype='float64'),
    'ForecastReturn': pd.Series(dtype='float64'),
    'i_USD': pd.Series(dtype='float64'),
    'OrderType': pd.Series(dtype='str'),
    'Exchange': pd.Series(dtype='str'),
    'AssetType': pd.Series(dtype='str')})
"""

¥


"\nLEDGER FORMAT\n    'OpenTimestamp': pd.Series(dtype='pd.datetime64[ns]'),\n    'UpdateTimestamp': pd.Series(dtype='pd.datetime64[ns]'),\n    'Asset': pd.Series(dtype='str'),\n    'Units': pd.Series(dtype='float64'),\n    'UnitWAP': pd.Series(dtype='float64'),\n    'TransactionIdxs': pd.Series(dtype='list'),\n    'AssetType': pd.Series(dtype='str'),\n    'CloseTimestamp': pd.Series(dtype='pd.datetime64[ns]'),\n    'Open': pd.Series(dtype='Bool')})\n\nTRADE FORMAT\n\n    'Timestamp': pd.Series(dtype='pd.datetime64[ns]'),\n    'Asset': pd.Series(dtype='str'),\n    'Units': pd.Series(dtype='float64'),\n    'UnitPriceUSD': pd.Series(dtype='float64'),\n    'OrderValueUSD': pd.Series(dtype='float64'),\n    'ForecastReturn': pd.Series(dtype='float64'),\n    'i_USD': pd.Series(dtype='float64'),\n    'OrderType': pd.Series(dtype='str'),\n    'Exchange': pd.Series(dtype='str'),\n    'AssetType': pd.Series(dtype='str')})\n"